In [1]:
import tensorflow as tf
import numpy as np

Simulate N by D data for logistic regression.

In [4]:
N = 20
D = 4
beta_true = np.random.uniform(-1, 1, 4).reshape((D, 1))
dataX = np.random.normal(0, 1, N * D).reshape(N, D)
prob = 1.0 / (1.0 + np.exp(-np.dot(dataX, beta_true)))
dataY = np.random.binomial(1, prob) * 1.0
print(dataX.shape)
print(dataY.shape)

(20, 4)
(20, 1)


Placeholding tensors are for variables. Their dimensions must be declared.

In [7]:
X = tf.placeholder('float', [None, D]) 
Y = tf.placeholder('float', [None, 1])

Variable is for parameters. The initial values are randomly drawn from Normal distribution.

In [6]:
beta = tf.Variable(tf.random_normal([D, 1], stddev=1.0))

Perceptron output with sigmoid activation is also predicted probability.

In [11]:
pred = 1.0 / (1.0 + tf.exp(-tf.matmul(X, beta)))

Binary cross entropy loss or negative log likelihood.

In [12]:
loss = -1.0 * (tf.reduce_sum(Y * tf.log(pred) + (1.0 - Y) * tf.log(1 - pred)))

**Ingredients for gradient descent**

In [13]:
grad = tf.gradients(loss, beta)

In [14]:
print(grad)

[<tf.Tensor 'gradients/MatMul_grad/MatMul_1:0' shape=(4, 1) dtype=float32>]


In [16]:
delta = tf.placeholder('float', [D, 1])
step = beta.assign_add(delta)

**Exercise 3**

In [18]:
step_size = 0.1
n_step = 10
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(n_step):
        # compute gradient
        g = sess.run(grad, feed_dict={X: dataX, Y: dataY})[0]
        # TODO
        print('Current beta: {}'.format(beta.eval().reshape(D)))

Current beta: [ 0.22412297  0.6724183   0.5185366  -1.5185701 ]
Current beta: [ 0.20883518  0.1961411   0.6965184  -1.274985  ]
Current beta: [ 0.15965374 -0.18006286  0.8309034  -1.0933487 ]
Current beta: [ 0.09479149 -0.47391707  0.9306526  -0.9733024 ]
Current beta: [ 0.02802605 -0.70508134  1.0055956  -0.9020382 ]
Current beta: [-0.03353162 -0.8902539   1.0634917  -0.8643212 ]
Current beta: [-0.08741981 -1.0418029   1.1096554  -0.8480482 ]
Current beta: [-0.13351047 -1.1683577   1.1475556  -0.8450162 ]
Current beta: [-0.1725963  -1.2758726   1.1794589  -0.84999675]
Current beta: [-0.20571366 -1.3685105   1.2068775  -0.8597198 ]
